In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install detoxify
!pip3 install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.7 MB/s 
     |████████████████████████████████| 4.9 MB 60.3 MB/s 
     |████████████████████████████████| 182 kB 76.3 MB/s 
     |████████████████████████████████| 6.6 MB 65.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 37.0 MB/s 


In [3]:
from detoxify import Detoxify

%load_ext autotime


import pandas as pd
import numpy as np

import re
# from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()


# lemmatizer = WordNetLemmatizer()
# punctuations = """!"$%()*+,*/:#»«'";“<=>?[\]^`”{.}~"""
punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)
    
    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT',' ',row)
    row = row.lower()
    
    row=re.sub("@[A-Za-z0-9_]+","",row)
    row=re.sub("http\S+|www.\S+","",row)

    row = re.sub(r'<.*?>', ' ', row)
    row = re.sub('\w*\d\w*','',row)
    row=re.sub("&amp","&",row)
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    row=re.sub('-',' ',row)
    # no_punct = ""
    # for char in row:
    #     if char not in punctuations:
    #         no_punct = no_punct + char
    # no_punct = no_punct.strip()
    row = re.sub('\s+',' ',row)
    return row

model=Detoxify('original',device='cuda')

my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


  0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

time: 1min 45s (started: 2022-12-21 15:29:03 +00:00)


In [16]:
model=Detoxify('unbiased',device='cpu')

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_debiased-c7548aa0.ckpt


  0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

time: 2min 12s (started: 2022-12-21 15:35:36 +00:00)


In [22]:
model.predict('Usefulness')

{'toxicity': 0.00041253906,
 'severe_toxicity': 1.2790151e-06,
 'obscene': 2.1400881e-05,
 'identity_attack': 7.602059e-05,
 'insult': 0.00010965843,
 'threat': 2.136811e-05,
 'sexual_explicit': 1.2471319e-05}

time: 63.3 ms (started: 2022-12-21 15:45:54 +00:00)


In [ ]:
import os
discords = os.listdir(my_dir+'discord')
subreddits = os.listdir(my_dir+'reddit')

time: 2.84 ms (started: 2022-11-24 16:13:17 +00:00)


In [ ]:
def embed(model, sentences):
    """
    wrapper function for generating message embeddings
    """
    hate_arr_i = model.predict(list(sentences))['toxicity']

    return hate_arr_i

def split_embed(model_, all_sentences_, n_partitions_=20):
    all_sentences_ = np.array_split(all_sentences_, indices_or_sections = n_partitions_)
    for i, new_sentences in enumerate(all_sentences_):
        if i==0:
            print(f'{i+1}',flush=True,end='.')
            all_hate_arr = embed(model_, new_sentences)
        if i>0:
            print(f'{i+1}',flush=True,end='.')
            new_hate_arr = embed(model_, new_sentences)
            all_hate_arr = np.concatenate((all_hate_arr,new_hate_arr))
    return all_hate_arr

time: 2.21 ms (started: 2022-11-24 16:03:01 +00:00)


In [ ]:
model.predict('this is outrageous')['toxicity']

0.04202426

time: 71.2 ms (started: 2022-11-24 19:21:23 +00:00)


In [ ]:
platform = 'reddit'
# platform = 'discord'

if platform == 'reddit':
  text_col = 'comment_text'
  author_col = 'author'
  platform_files = subreddits


if platform == 'discord':
  text_col = 'content'
  author_col = 'author_username'
  platform_files = discords

for file_name in platform_files:

  print(f'\nbegining with {file_name} {platform}')

  data=pd.read_csv(f'{my_dir}{platform}/{file_name}',lineterminator='\n')
  print(f'{file_name} has {len(data)} data')
  data[text_col]=data[text_col].astype(str)

# reddit_x[reddit_x['comment_text'].duplicated()]

  # data[text_col + '_cleaned']=data[text_col].progress_apply(lambda x: clean_text_local(x))

  # hate_arr = split_embed(model, data[text_col],n_partitions_=10000)
  # data['Toxic_BERT']=hate_arr

  data['Toxic_BERT']=data[text_col].progress_apply(lambda x: model.predict(str(x))['toxicity'])

  data.to_csv(f'{my_dir}{platform}/{file_name}',index=None)


begining with dankmemes_subreddit_hate.csv reddit
dankmemes_subreddit_hate.csv has 1800000 data
1.

RuntimeError: ignored

time: 9.2 s (started: 2022-11-24 16:03:36 +00:00)
